# Strassen's Algorithm for Faster Multiplication

Miles Milosevich






































## Introduction

Strassen's algorithm serves to increase the speed of matrix multiplication through recursion. Using some specific tricks, the number of multiplication operations required to perform matrix multiplication of $2x2$ matrices are reduced from $8$ to $7$. Since the multiplication operation is fairly computationally expensive, reducing the number of multiplications required reduces computational complexity, and increases speed.

## Algorithm

Let us consider the "Divide and Conquer" strategy for matrix multiplication. In this, matrices are recursively divided into smaller quadrants, or block matrices, until only 2x2 matrices remain, which are multiplied. 

$$
\begin{bmatrix} 
C_{00} & C_{01} \\
C_{10} & C_{11} 
\end{bmatrix}
=
\begin{bmatrix} 
A_{00} & A_{01} \\
A_{10} & A_{11} 
\end{bmatrix}
\cdot
\begin{bmatrix} 
B_{00} & B_{01} \\
B_{10} & B_{11} 
\end{bmatrix}
$$

The formula for calculating each block of $C$:
$$
\begin{align*}
C_{00} &= A_{00} \cdot B_{00} + A_{01} \cdot B_{10} \\
C_{01} &= A_{00} \cdot B_{01} + A_{01} \cdot B_{11} \\
C_{10} &= A_{10} \cdot B_{00} + A_{11} \cdot B_{10} \\
C_{11} &= A_{10} \cdot B_{01} + A_{11} \cdot B_{11}
\end{align*}
$$

For recursive block matrix multiplication, matrices are padded until they have a dim size of a power of 2; as such, matrices can be subdivided to be entirely composed of 2x2 block matrices. This padding can be removed after multiplication.

#### Operations and Complexity
- **Number of Multiplications**: $ 8n^{2} $ where $n$ is the dimension of the matrix, assuming no reductions or further optimizations.
- **Time Complexity**: $ T(n) = 8T\left(\frac{n}{2}\right) + O(n^2) $

The matrix multiplication operation is expensive, motivating the creation of an algorithm using fewer matrix multiplication operations. Strassen's algorithm implements this; in the above Naive approach, 8 multiplication operations are used, while is Strassen's below, only 7 are used.

$$
\begin{bmatrix} 
C_{00} & C_{01} \\
C_{10} & C_{11} 
\end{bmatrix}
=
\begin{bmatrix}
M_1 + M_4 - M_5 + M_7 & M_3 + M_5 \\
M_2 + M_4 & M_1 - M_2 + M_3 + M_6
\end{bmatrix}
$$

Where the $M_i$ terms are computed as:
$$
\begin{align*}
M_1 &= (A_{00} + A_{11}) \cdot (B_{00} + B_{11}) \\
M_2 &= (A_{10} + A_{11}) \cdot B_{00} \\
M_3 &= A_{00} \cdot (B_{01} - B_{11}) \\
M_4 &= A_{11} \cdot (B_{10} - B_{00}) \\
M_5 &= (A_{00} + A_{01}) \cdot B_{11} \\
M_6 &= (A_{10} - A_{00}) \cdot (B_{00} + B_{01}) \\
M_7 &= (A_{01} - A_{11}) \cdot (B_{10} + B_{11})
\end{align*}
$$

#### Operations and Complexity
- **Number of Multiplications**: $7^{\log_2(n/t)} $
- **Time Complexity**: $ T(n) = 7T\left(\frac{n}{2}\right) + O(n^2) $


## Implementation

Strassen's algorithm is known for only being beneficial in high-dimensional matrices of at least 512. In order process multiplication at this dimensionality, multiprocessing is implemented using the **multiprocess** library, a multiprocessing library capable of serializing constructs the builtin **multiprocessing** cannot, such as lambda functions and complex structures.

For full code, see BlockMMultFactory (an object for creating matrix multiplication operators) for implementation details regarding multiprocessing and generalized matrix multiplication.

Typically, implementations will use a threshold to switch to standard matrix multiplicatin, wherein performing strassen multiplication on matrices smaller than this threshold will switch to normal matrix multiplication. This behavior is implemented, but not used in the following example.

In [1]:
from blockmmult import BlockMMultFactory
import numpy as np


def create_strassen_callable(threshold=1, max_thread_depth=2):
    
    #select A and B quadrants for recursive block multiplication calls
    scatter_targets = [
        ((lambda A, B : A[0][0] + A[1][1]), (lambda A, B : B[0][0] + B[1][1])),
        ((lambda A, B : A[1][0] + A[1][1]), (lambda A, B : B[0][0])),
        ((lambda A, B : A[0][0]), (lambda A, B : B[0][1] - B[1][1])),
        ((lambda A, B : A[1][1]), (lambda A, B : B[1][0] - B[0][0])),
        ((lambda A, B : A[0][0] + A[0][1]), (lambda A, B : B[1][1])),
        ((lambda A, B : A[1][0] - A[0][0]), (lambda A, B : B[0][0] + B[0][1])),
        ((lambda A, B : A[0][1] - A[1][1]), (lambda A, B : B[1][0] + B[1][1])),
    ]
    
    # how will results from the above be summated?
    gather_tasks = [
        (
        #C11 & C12
            (lambda M : M[0] + M[3] - M[4] + M[6]),
            (lambda M : M[2] + M[4]),
        ),(
        #C21 & C22
            (lambda M : M[1] + M[3]),
            (lambda M : M[0] - M[1] + M[2] + M[5]),
        )
    ]
    def strassen_mult_calls(n, threshold):
        return 7 ** (np.log2(n / threshold))
        
    return BlockMMultFactory(scatter_targets, gather_tasks, threshold, max_thread_depth, strassen_mult_calls)

def create_naive_callable(threshold=1, max_thread_depth=2):

    scatter_targets = [
        ((lambda A, B : A[0][0]), (lambda A, B : B[0][0])),
        ((lambda A, B : A[0][1]), (lambda A, B : B[1][0])),
        ((lambda A, B : A[0][0]), (lambda A, B : B[0][1])),
        ((lambda A, B : A[0][1]), (lambda A, B : B[1][1])),
        ((lambda A, B : A[1][0]), (lambda A, B : B[0][0])),
        ((lambda A, B : A[1][1]), (lambda A, B : B[1][0])),
        ((lambda A, B : A[1][0]), (lambda A, B : B[0][1])),
        ((lambda A, B : A[1][1]), (lambda A, B : B[1][1]))
    ]
    gather_tasks = [
        (
        #C11 & C12
            (lambda M : M[0] + M[1]),
            (lambda M : M[2] + M[3]),
        ),(
        #C21 & C22
            (lambda M : M[4] + M[5]),
            (lambda M : M[6] + M[7]),
        )
    ]
    def naive_mult_calls(n, t):
        # Each split increases the number of operations by a factor of 8, and we split log2(n/t) times
        return 8 ** np.log2(n / t) * (t ** 3)
    
    return BlockMMultFactory(scatter_targets, gather_tasks, threshold, max_thread_depth, naive_mult_calls)


n = 4
threshold = 1
max_thread_depth = 2

Strassen_Mult_Operator = create_strassen_callable(threshold, max_thread_depth)
Naive_Block_Mult_Operator = create_naive_callable(threshold, max_thread_depth)

A = np.random.rand(n,n)
B = np.random.rand(n,n)

C_Strassen = Strassen_Mult_Operator(A, B, verbose=2)
C_Naive = Naive_Block_Mult_Operator(A, B, verbose=2)


print("Result of Strassen Matrix Multiplication:\n", C_Strassen)
print("Result of Naive Matrix Multiplication:\n", C_Naive)

print("Expected result:\n", np.dot(A,B))

assert np.allclose(C_Strassen,np.dot(A,B))
assert np.allclose(C_Naive,np.dot(A,B))

IntProgress(value=0, max=49)

Total multiplications: 49


IntProgress(value=0, max=64)

Total multiplications: 64
Result of Strassen Matrix Multiplication:
 [[1.31168784 0.67590843 0.94040262 1.09402739]
 [1.00585111 0.86113725 0.54883298 1.11767388]
 [1.1653255  0.8364104  0.616314   1.24503956]
 [1.08346714 1.32342191 0.68733291 1.4313166 ]]
Result of Naive Matrix Multiplication:
 [[1.31168784 0.67590843 0.94040262 1.09402739]
 [1.00585111 0.86113725 0.54883298 1.11767388]
 [1.1653255  0.8364104  0.616314   1.24503956]
 [1.08346714 1.32342191 0.68733291 1.4313166 ]]
Expected result:
 [[1.31168784 0.67590843 0.94040262 1.09402739]
 [1.00585111 0.86113725 0.54883298 1.11767388]
 [1.1653255  0.8364104  0.616314   1.24503956]
 [1.08346714 1.32342191 0.68733291 1.4313166 ]]


## Performance Benchmarking

Here we compare the performance between the two algorithms in terms of time, number of multiplications, and error.

Time measured in seconds. Error measured as froebenius norm of the difference between the results from custom matrix operation (naive or strassen) versus numpy's matrix multiplication implementation.

In [3]:
import pandas as pd

In [2]:
import timeit

def measure_performance(size, num_trials, threshold=1, max_thread_depth=2, verbose=1):
    
    Strassen_Mult_Operator = create_strassen_callable(threshold, max_thread_depth)
    Naive_Block_Mult_Operator = create_naive_callable(threshold, max_thread_depth)

    strassen_times = []
    naiveblock_times = []
    strassen_counts = []
    naiveblock_counts = []
    strassen_errs = []
    naiveblock_errs = []
    
    for _ in range(num_trials):
        A = np.random.rand(size,size)
        B = np.random.rand(size,size)
        res_true = np.dot(A,B)
                
        # Strassen
        start_time = timeit.default_timer()
        res, count = Strassen_Mult_Operator(A, B, verbose=verbose, return_count=True)
        strassen_times.append(timeit.default_timer() - start_time)
        strassen_counts.append(count)
        strassen_errs.append(np.linalg.norm(res - res_true))
        
        # Naive
        start_time = timeit.default_timer()
        res, count = Naive_Block_Mult_Operator(A, B, verbose=verbose, return_count=True)
        naiveblock_times.append(timeit.default_timer() - start_time)
        naiveblock_counts.append(count)
        naiveblock_errs.append(np.linalg.norm(res - res_true))
        

    return np.mean(strassen_times), np.mean(naiveblock_times), np.mean(strassen_counts), np.mean(naiveblock_counts), np.mean(strassen_errs), np.mean(naiveblock_errs)

max_power_of_2_dim = 11
sizes = [2**n for n in range(2, max_power_of_2_dim+1)]
num_trials = 1
threshold = 1
max_thread_depth = 3
verbose = 1

data = []
for size in sizes:
    print('measuring performance for mult on matrices of size ', size)
    performance = measure_performance(size, num_trials, threshold, max_thread_depth, verbose)
    data.append([size, *performance])

df = pd.DataFrame(data, columns=['Matrix Size', 'Strassen Time', 'Naive Time', 'Strassen Multiplications', 'Naive Multiplications', 'Strassen Error', 'Naive Error'])

measuring performance for mult on matrices of size  4


IntProgress(value=0, max=49)

IntProgress(value=0, max=64)

measuring performance for mult on matrices of size  8


IntProgress(value=0, max=343)

...

  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
 

  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/mp_safe.py", line 48, in increment
    with s

  File "/home/milesmilos/python/lib/python3.9/site-packages/multiprocess/managers.py", line 1044, in __enter__
    return self._callmethod('acquire')
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) f

  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/python/lib/python3.9/site-packages/multiprocess/connection.py", line 253, in recv
    buf = self._recv_bytes()
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in n

  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in <listcomp>
    results = [t[0].mmult_recursive(*t[1:]) for t in next_mmult_args]
  File "/home/milesmilos/python/lib/python3.9/site-packages/multiprocess/managers.py", line 810, in _callmethod
    kind, result = conn.recv()
  File "/home/milesmilos/Documents/courses/MATH315/BlockMult/src/blockmmult.py", line 83, in mmult_recursive
    results = [t[0].mmult_recursive(*

In [5]:
df = pd.DataFrame(data, columns=['Matrix Size', 'Strassen Time', 'Naive Time', 'Strassen Multiplications', 'Naive Multiplications', 'Strassen Error', 'Naive Error'])

In [7]:
df = pd.read_csv('calced_to_512.csv', index_col=0)

In [8]:
from IPython.display import display, HTML
display(HTML(df.to_html()))

,Matrix Size,Strassen Time,Naive Time,Strassen Multiplications,Naive Multiplications,Strassen Error,Naive Error
0,4,0.592260,0.832173,49.0,64.0,1.023575e-15,4.299875e-16
1,8,4.886819,6.596321,343.0,512.0,1.172429e-14,1.961045e-15
2,16,6.526534,9.588517,2401.0,4096.0,6.060231e-14,8.859552e-15
3,32,17.981483,33.246071,16807.0,32768.0,4.703205e-13,4.668643e-14
4,64,99.518964,195.497321,117649.0,262144.0,3.289781e-12,2.333527e-13
5,128,588.320001,1490.888218,823543.0,2097152.0,2.258010e-11,1.269378e-12
6,256,4047.632766,12010.782755,5764801.0,16777216.0,1.341155e-10,6.793130e-12
7,512,32701.971604,107643.343723,40353607.0,134217728.0,1.052728e-09,2.014630e-11


In [7]:
#df.to_csv('calced_to_512.csv')

## Derivation & Proof

Here, I present several ideas for deriving/proving Strassen.

### Method 1: $A$ Matrix Decomposition

#### Step 1: Equivalent Matrix Multiplication
The reduction to block matrices is useful computationally for implementing recursion. Mathematically, it allows for proving an algorithm like Strassen's works for all matrices of dimensionality of a power of 2.

As such, we can start by reformulating the problem of multiplying two $2 \times 2$ matrices into a matrix-vector product problem. This step abstracts the matrix multiplication into a more linear algebra-focused perspective, aiding in the simplification and manipulation of terms.

The multiplication of two matrices, like in
$$ \begin{pmatrix} a & b \\ c & d \end{pmatrix} \times \begin{pmatrix} e & f \\ g & h \end{pmatrix} $$
is the same as computing
$$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & c & 0 \\
0 & d & 0 & d
\end{pmatrix}
\times
\begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix}
$$

We reshape this multiplication problem into a matrix-vector product by reorganizing the elements of $A$ and $B$ into new configurations that expose their linear interactions more clearly.

#### Step 2: Decompose and Rearrange
Seek a decomposition of the $4 \times 4$ matrix where each component (let's call these $M_i$) involves simpler, potentially rank-one matrices that are easier to handle. The goal is to express the large matrix as a sum of products where each product involves fewer independent variables: $\ell_1 M_1 + \cdots + \ell_7 M_7$, where $\ell_i$ is a linear combination of $a,b,c,d$. As such, the product we are looking for is 
$$
\sum_{i=1}^7 \ell_i M_i \begin{pmatrix} c\\d\\e\\f \end{pmatrix} =
\sum_{i=1}^7 \ell_i x_i y_i^T \begin{pmatrix} c\\d\\e\\f \end{pmatrix} =
\sum_{i=1}^7 \ell_i r_i x_i,
$$
where $r_i$ is a linear combination of $e,f,g,h$, where $r_i$ is a linear combination of $e,f,g,h$, and each entry of the product matrix is some linear combination of the products $\ell_i,r_i$. This part involves creative insight to "guess" beneficial decompositions.

For example, decompose the matrix into parts where each contributes separately to the output matrix with minimal overlap in variables used:
$$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & d & 0 \\
0 & c & 0 & d
\end{pmatrix}
\rightarrow \text{Decomposed into several simpler matrices}
$$

For example, $$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & d & 0 \\
0 & c & 0 & d
\end{pmatrix} =
\begin{pmatrix}
a & 0 & a & 0 \\
0 & 0 & 0 & 0 \\
a & 0 & a & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & d & 0 & d \\
0 & 0 & 0 & 0 \\
0 & d & 0 & d
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & a-d & 0 & b-d \\
c-a & 0 & d-a & 0 \\
0 & c-d & 0 & 0
\end{pmatrix}
$$
This results in a mess, which we try to fix by "flipping" the inner square:
$$
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & a-d & 0 & b-d \\
c-a & 0 & d-a & 0 \\
0 & c-d & 0 & 0
\end{pmatrix} =
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & a-d & a-d & 0 \\
0 & d-a & d-a & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & 0 & d-a & b-d \\
c-a & a-d & 0 & 0 \\
0 & c-d & 0 & 0
\end{pmatrix}
$$
Since $d-a = (b-a)-(b-d)$ and $a-d = (c-d)-(c-a)$, it is easy to represent the last matrix as a sum of four rank one matrices:
$$
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & 0 & d-a & b-d \\
c-a & a-d & 0 & 0 \\
0 & c-d & 0 & 0
\end{pmatrix} =
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & 0 & b-a & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & d-b & b-d \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & c-d & 0 & 0 \\
0 & c-d & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
c-a & a-c & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix}
$$
In total, we obtain the following representation:
$$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & d & 0 \\
0 & c & 0 & d
\end{pmatrix} =
\begin{pmatrix}
a & 0 & a & 0 \\
0 & 0 & 0 & 0 \\
a & 0 & a & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & d & 0 & d \\
0 & 0 & 0 & 0 \\
0 & d & 0 & d
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & a-d & a-d & 0 \\
0 & d-a & d-a & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & 0 & b-a & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & d-b & b-d \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & c-d & 0 & 0 \\
0 & c-d & 0 & 0
\end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
c-a & a-c & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix}
$$

If we multiply $B$ to each of these matrices, to check, we find

$$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & d & 0 \\
0 & c & 0 & d
\end{pmatrix}
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} =
\begin{pmatrix}
a & 0 & a & 0 \\
0 & 0 & 0 & 0 \\
a & 0 & a & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} 
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & d & 0 & d \\
0 & 0 & 0 & 0 \\
0 & d & 0 & d
\end{pmatrix} 
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & a-d & a-d & 0 \\
0 & d-a & d-a & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} 
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} +
\begin{pmatrix}
0 & 0 & b-a & 0 \\
0 & 0 & b-a & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} 
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & d-b & b-d \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix} 
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & c-d & 0 & 0 \\
0 & c-d & 0 & 0
\end{pmatrix} 
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} +
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
c-a & a-c & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix}
\cdot \begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix} 
$$
$$ =
\begin{pmatrix} a(g + e) \\ 0 \\ a(g + e) \\ 0 \end{pmatrix}^T +
\begin{pmatrix} 0 \\ d(f+h) \\ 0 \\ d(f+h) \end{pmatrix}^T +
\begin{pmatrix} 0 \\ (f+g)(a-d) \\ 0 \\ (f+g)(a-d) \end{pmatrix}^T +
\begin{pmatrix} g(b-a) \\ g(b-a) \\ 0\\ 0 \end{pmatrix}^T +
\begin{pmatrix} 0 \\ g(d-b) + h(b-d) \\ 0 \\ 0 \end{pmatrix}^T +
\begin{pmatrix} 0 \\ 0 \\ f(c-d) \\ f(c-d) \end{pmatrix}^T +
\begin{pmatrix} 0 \\ 0 \\ f(a-c) + e(c-a) \\ 0 \end{pmatrix}^T +
$$

This example did not work, as we still have >7 matrix operations (9 in this case), but it exemplifies how one could derive the strassen algorithm. If we had chosen some decomposition where the terms $g(d-b) + h(b-d)$ and $f(a-c) + e(c-a)$ could properly un-distribute, we would have 7 matrix multiplication operations.

(from Yuval, 1978; example from stackexchange)

But this is a matter of luck/brute force. How did Strassen derive this?

### Method 2: Bilinear Matrices

Instead of performing equivalence of matrices for matrix multiplication, Strassen likely thought about this problem in a different way. 

We could think of the bilinear matrix representing matrix multiplication as:

$$
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}
$$

This matrix represents how the multiplied components of two $2x2$ matrix summed for each element in the output, $C$ (as in, row 1 represents $A_{00}B_{00}$, row 2 represents $A_{00}B_{01}$, $\ldots$ (the index of this matrix, in binary, represents the indices of $A$ and $B$ in this case). **Each column vector represents the vectors required to span the space of matrix multiplication**.

We can take advantage of the structure of this matrix to implement a rule to keep track of the number of multiplication operations taking place: Each bilinear matrix must contain only one non-zero value, or it contains non-zero values at row indices corresponding with matrix multiplications which are the result of matrix distribution; as in, if matrix $M_p$ contains a $1$ at the row corresponding to $A_{ij}B_{kl}$, it either contains no other elements, or the other elements are the result of distribution $(A_{ij} + \ldots)(B_{jl} + \ldots)$. Or in other words, other elements in the matrix must either correspond to a row containing $A_{ij}$ or $B_{kl}$, or, for example $A_{xy}B_{yz} \in M_p$, must be from distribution $(A_{ij} + A_{wx} + \ldots)(B_{kl} + B_{yz} + \ldots)$ (aka $M_p$ also contains $A_{jk}B{yz}$ and $A_{wx}B_{kl}$)

By defining this, we restrict 1 matrix multiplication operation per matrix.

The bilinear matrix representing matrix multiplication then becomes the sum of 8 matrices:

$$
M_1\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_2\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_3\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_4\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_5\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_6\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_7\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}+M_8\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}
$$
Which for the remainder of this paper, will be represented as 
$$
\begin{pmatrix}
M_1 & 0 & 0 & 0 \\
0 & M_2 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
M_3 & 0 & 0 & 0 \\
0 & M_4 & 0 & 0 \\
0 & 0 & M_5 & 0 \\
0 & 0 & 0 & M_6 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & M_7 & 0 \\
0 & 0 & 0 & M_8 \\
\end{pmatrix}
$$
(where it is implied the coefficients are all equal to eachother = 1)

As such, the dimensionality of the minimum spanning basis of matrix multiplication is shown to be 8... or is it?

Strassen provides the following solution to this problem:

$$
\begin{pmatrix}
M_1 & 0 & 0 & M_1 - M_6 \\
0 & M_3 & 0 & M_3 - M_6 \\
0 & 0 & 0 & 0 \\
M_1 - M_5 & M_5 - M_3 & 0 & M_1 - M_3 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
M_7 & 0 & 0 & 0 \\
M_7 - M_5 & M_5 & 0 & 0 \\
0 & 0 & M_2 & M_6 - M_2 \\
0 & 0 & 0 & M_6 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
M_1 - M_4 & 0 & M_2 - M_4 & M_1 - M_2 \\
0 & 0 & 0 & 0 \\
M_4 - M_7 & 0 & M_4 & 0 \\
M_1 - M_7 & 0 & 0 & M_1 \\
\end{pmatrix}
$$

As we mentioned, since each coefficient is 1, we see how Strassen uses null pairs (with each element of each pair in different matrices) such that, when this condition is removed, we return to the bilinear representation of matrix multiplication. Since only 7 matrices are used for this solution, instead of the naive 8 matrices, Strassen's solition reduces the dimensionality of the minimum spanning basis to be 7 rather than 8. Strassen's solution is not unique in this case. 

Finding a way to represent this condition with linear algebra is the next step of proving that 7 is the dimensionality of the minimum spanning basis. How can we do this?

Although this does not prove 7 is the minimum, we could create a secondary matrix which maps each $ijkl$ index in relation to the others, with a $1$ representing a positive relationship, and a $-1$ representing a negative relationship (for instead, the relationship between $ijkl$ and $ijyz$ is $1$, while the relationship between $ijkl$ and $wxyz$ is $-1$). With this, we could use a layered Hopfield network, each layer representing one of $M1$ through $M8$ matrices. This RNN would use the weight matrix I just specified, along with a secondary matrix containing conditionless bilinear matrix representing matrix multiplication, with higher weight. Some other OR algorithm could work as well.

I have not thought through the exact dimensionality of each matrix required for using this network, and exactly how the $ijkl$ mapping should work, but so that I turn this project in before its too late, I leave this as an excercise for the reader.

## Discussion

Although Strassen's algorithm clearly outperformed the Naive Block multiplication, Strassen's algorithm is not often used in practice. Better methods exist for sparse matrices, and most matrix multiplication occurs in the smaller dimensions where its greater overhead outweighs its increased error accumulation.

Strassen's algorithm remains unproven as the minimum number of matrix multiplications required. Perhaps with a proper definition of the problem space with linear algebra, including the conditions required for components to be included in a matrix, a proper proof might be presented, or it might be found that a smaller number of multiplications could be required. Maybe it would take the form of a matrix using something akin to $(A_{ij} + A_{kl} + A_{il}) \cdot (B_{xy} + B_{wx})$?

## References

Gideon Yuval,
A simple proof of Strassen's result,
Information Processing Letters,
Volume 7, Issue 6,
1978,
Pages 285-286,
ISSN 0020-0190,
https://doi.org/10.1016/0020-0190(78)90018-2.
(https://www.sciencedirect.com/science/article/pii/0020019078900182)
Keywords: Strassen's method; matrix multiplication; n2.81
    
    
https://cs.stackexchange.com/questions/130022/what-is-the-intuition-behind-strassens-algorithm

https://en.wikipedia.org/wiki/Strassen_algorithm